In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_excel('/content/drive/MyDrive/Datasets/final_mutation(proteomics).xlsx',index_col=0)
train.head()

,cell_name,MECOM_mut,MED23_mut,MGA_mut,MLL_mut,MLL3_mut,MLLT4_mut,MYH9_mut,NF1_mut,NF2_mut,NSD1_mut,NUP98_mut,PIK3CA_mut,PIK3R1_mut,POLR2B_mut,PTEN_mut,SACS_mut,SMARCA4_mut,STAG2_mut,SVEP1_mut,TCF12_mut,TP53_mut,TRIO_mut,ZFP36L1_mut,AHNAK_mut,AKAP9_mut,APC_mut,ARFGEF2_mut,ARID1A_mut,ARID1B_mut,ARID4B_mut,ASH1L_mut,ASPM_mut,ASXL2_mut,ATM_mut,ATR_mut,BAP1_mut,BNC2_mut,BRAF_mut,BRCA1_mut,...,cnaBRCA26,cnaBRCA27,cnaBRCA28,cnaBRCA29,cnaBRCA30,cnaBRCA31,cnaBRCA32,cnaBRCA33,cnaBRCA34,cnaBRCA35,cnaBRCA36,cnaBRCA37,cnaBRCA38,cnaBRCA39,cnaBRCA40,cnaBRCA41,cnaBRCA42,cnaBRCA43,cnaBRCA44,cnaBRCA45,cnaBRCA46,cnaBRCA47,cnaBRCA48,cnaBRCA49,cnaBRCA50,cnaBRCA51,cnaBRCA52,cnaBRCA53,cnaBRCA54,cnaBRCA55,cnaBRCA56,cnaBRCA57,cnaBRCA58,cnaBRCA59,cnaBRCA60,cnaBRCA61,cnaBRCA62,cnaBRCA63,cnaBRCA64,cnaBRCA65
0,AU565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,HCC1143,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
16,HCC1187,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
17,HCC1395,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
20,HCC1500,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,1,0,0,1,1,1,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0


In [ ]:
target=train['cell_name']

In [ ]:
train.drop(['cell_name'], axis=1, inplace=True)
train.head()

,MECOM_mut,MED23_mut,MGA_mut,MLL_mut,MLL3_mut,MLLT4_mut,MYH9_mut,NF1_mut,NF2_mut,NSD1_mut,NUP98_mut,PIK3CA_mut,PIK3R1_mut,POLR2B_mut,PTEN_mut,SACS_mut,SMARCA4_mut,STAG2_mut,SVEP1_mut,TCF12_mut,TP53_mut,TRIO_mut,ZFP36L1_mut,AHNAK_mut,AKAP9_mut,APC_mut,ARFGEF2_mut,ARID1A_mut,ARID1B_mut,ARID4B_mut,ASH1L_mut,ASPM_mut,ASXL2_mut,ATM_mut,ATR_mut,BAP1_mut,BNC2_mut,BRAF_mut,BRCA1_mut,BRCA2_mut,...,cnaBRCA26,cnaBRCA27,cnaBRCA28,cnaBRCA29,cnaBRCA30,cnaBRCA31,cnaBRCA32,cnaBRCA33,cnaBRCA34,cnaBRCA35,cnaBRCA36,cnaBRCA37,cnaBRCA38,cnaBRCA39,cnaBRCA40,cnaBRCA41,cnaBRCA42,cnaBRCA43,cnaBRCA44,cnaBRCA45,cnaBRCA46,cnaBRCA47,cnaBRCA48,cnaBRCA49,cnaBRCA50,cnaBRCA51,cnaBRCA52,cnaBRCA53,cnaBRCA54,cnaBRCA55,cnaBRCA56,cnaBRCA57,cnaBRCA58,cnaBRCA59,cnaBRCA60,cnaBRCA61,cnaBRCA62,cnaBRCA63,cnaBRCA64,cnaBRCA65
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,1,1,1,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0


In [ ]:
train_scaled = minmax_scale(train, axis = 0)

In [ ]:
ncol = train_scaled.shape[1]
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Deep Autoencoder

# this is the size of our encoded representations
encoding_dim = 32

# this is our input placeholder; 784 = 28 x 28
input = Input(shape = (ncol, ))

my_epochs = 50

# "encoded" is the encoded representation of the inputs
encoded = Dense(encoding_dim * 4, activation='relu')(input)
encoded = Dense(encoding_dim * 2, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
decoded = Dense(encoding_dim * 4, activation='relu')(decoded)
decoded = Dense(ncol, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input, decoded)

# Separate Encoder model

# this model maps an input to its encoded representation
encoder = Model(input, encoded)

# Separate Decoder model

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))
# retrieve the layers of the autoencoder model
decoder_layer1 = autoencoder.layers[-3]
decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

# Train to reconstruct MNIST digits

# configure model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# prepare input data
#(x_train, _), (x_test, y_test) = mnist.load_data()
x_train, x_test, y_train, y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))
# normalize all values between 0 and 1 and flatten the 28x28 images into vectors of size 784
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

# Train autoencoder for 50 epochs

autoencoder.fit(x_train, x_train, epochs=my_epochs, batch_size=8, shuffle=True, validation_data=(x_test, x_test),
                verbose=2)

# after 100 epochs the autoencoder seems to reach a stable train/test lost value

# Visualize the reconstructed encoded representations

# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)


encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train.head()

K.clear_session()

(10, 128)
(2, 128)
Epoch 1/50
2/2 - 1s - loss: 0.6945 - val_loss: 0.6955 - 1s/epoch - 586ms/step
Epoch 2/50
2/2 - 0s - loss: 0.6945 - val_loss: 0.6955 - 31ms/epoch - 16ms/step
Epoch 3/50
2/2 - 0s - loss: 0.6944 - val_loss: 0.6955 - 24ms/epoch - 12ms/step
Epoch 4/50
2/2 - 0s - loss: 0.6944 - val_loss: 0.6955 - 28ms/epoch - 14ms/step
Epoch 5/50
2/2 - 0s - loss: 0.6944 - val_loss: 0.6955 - 24ms/epoch - 12ms/step
Epoch 6/50
2/2 - 0s - loss: 0.6944 - val_loss: 0.6955 - 26ms/epoch - 13ms/step
Epoch 7/50
2/2 - 0s - loss: 0.6944 - val_loss: 0.6955 - 25ms/epoch - 12ms/step
Epoch 8/50
2/2 - 0s - loss: 0.6944 - val_loss: 0.6955 - 30ms/epoch - 15ms/step
Epoch 9/50
2/2 - 0s - loss: 0.6944 - val_loss: 0.6955 - 29ms/epoch - 15ms/step
Epoch 10/50
2/2 - 0s - loss: 0.6944 - val_loss: 0.6955 - 32ms/epoch - 16ms/step
Epoch 11/50
2/2 - 0s - loss: 0.6944 - val_loss: 0.6955 - 23ms/epoch - 12ms/step
Epoch 12/50
2/2 - 0s - loss: 0.6944 - val_loss: 0.6955 - 35ms/epoch - 18ms/step
Epoch 13/50
2/2 - 0s - loss: 0.

In [ ]:
encoded_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,0.120454,0.127926,0.0,0.0,0.000000,0.193590,0.000000,0.0,0.0,0.223327,0.154020,0.000000,0.102905,0.037008,0.000000,0.106452,0.000000,0.070097,0.0,0.009010,0.0,0.009496,0.214605,0.115544,0.003563,0.041948,0.000000,0.117040,0.032682,0.0,0.114352,0.000000
1,0.349228,0.141427,0.0,0.0,0.072363,0.178180,0.064141,0.0,0.0,0.000000,0.000000,0.111016,0.004723,0.092346,0.005280,0.000000,0.000000,0.125273,0.0,0.192122,0.0,0.134140,0.051913,0.000000,0.227601,0.000000,0.000000,0.000000,0.067361,0.0,0.314656,0.000000
2,0.106099,0.079531,0.0,0.0,0.017179,0.000000,0.027715,0.0,0.0,0.000000,0.000000,0.000000,0.066143,0.006419,0.042245,0.032690,0.083960,0.000000,0.0,0.031854,0.0,0.045838,0.075555,0.000000,0.000000,0.000000,0.000000,0.066277,0.063918,0.0,0.017984,0.000000
3,0.361275,0.105729,0.0,0.0,0.151691,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.020968,0.189537,0.137460,0.116805,0.091188,0.070248,0.000000,0.0,0.182455,0.0,0.078981,0.131215,0.000000,0.145929,0.000000,0.109870,0.312561,0.000000,0.0,0.044058,0.069587
4,0.683590,0.512473,0.0,0.0,0.140701,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.469482,0.361556,0.218819,0.478536,0.223206,0.000000,0.0,0.259070,0.0,0.000000,0.091189,0.000000,0.000000,0.000000,0.372783,0.477388,0.018820,0.0,0.043959,0.000000
5,0.311975,0.068812,0.0,0.0,0.000000,0.109904,0.008936,0.0,0.0,0.000000,0.092165,0.039992,0.177332,0.000000,0.000000,0.036398,0.008012,0.000000,0.0,0.133298,0.0,0.145151,0.062330,0.000000,0.126165,0.000000,0.110263,0.036590,0.000000,0.0,0.181314,0.129823
6,0.399813,0.153993,0.0,0.0,0.000000,0.215970,0.025563,0.0,0.0,0.000000,0.268538,0.000000,0.306233,0.125426,0.077267,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.137550,0.134737,0.000000,0.006107,0.000000,0.147800,0.253165,0.049110,0.0,0.365697,0.156365
7,0.100782,0.178239,0.0,0.0,0.147906,0.158692,0.000000,0.0,0.0,0.006373,0.021568,0.000000,0.076648,0.000000,0.068027,0.206161,0.000000,0.000000,0.0,0.159852,0.0,0.043923,0.263763,0.077522,0.000000,0.000000,0.000000,0.038067,0.000000,0.0,0.146836,0.000000
8,0.455922,0.283306,0.0,0.0,0.017253,0.049533,0.036147,0.0,0.0,0.000000,0.082887,0.000000,0.321922,0.081874,0.000000,0.599739,0.163580,0.000000,0.0,0.136628,0.0,0.000000,0.000000,0.000000,0.047600,0.000000,0.049102,0.586666,0.000000,0.0,0.364630,0.191002
9,0.154334,0.000000,0.0,0.0,0.310956,0.094158,0.000000,0.0,0.0,0.188811,0.288000,0.000000,0.106691,0.009281,0.008707,0.076268,0.000000,0.002445,0.0,0.000000,0.0,0.000000,0.285782,0.151656,0.000000,0.195048,0.000000,0.023406,0.044173,0.0,0.405062,0.000000


In [ ]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 128)               16512 

In [ ]:
encoded_train = encoded_train.loc[:, (encoded_train != 0).any(axis=0)]
encoded_train = encoded_train.drop(columns=encoded_train.columns[encoded_train.eq(0).mean()>0.8])
encoded_train.shape

(12, 24)

In [ ]:
encoded_train.to_excel('/content/drive/MyDrive/Datasets/dae_mutation(proteomics).xlsx', index=False)